# LOICA Enzymes (in branch Consortia) demonstration

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

This notebook shows the novel classes Degrader and Producer, which belong to class Impactor that represents relationships between gene products on the level of enzymes. These new features are part of git branch Consortia.

In [ ]:
def growth_rate(t):
    return 0 #lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return 1 #lc.gompertz(t, 0.01, 1, 1, 1)

## Degrader

First, create metabolism and genetic network, then create strain with these parameters.

In [ ]:
metab1 = lc.SimulatedMetabolism("Simulated metabolism 1", biomass, growth_rate)
genetic_n1 = lc.GeneticNetwork()
test_strain_1= lc.Strain('Test strain', genetic_n1, metab1)

Degrader takes the name (use enzyme function name), enzyme is GeneProduct which has the active site, substrate is the GeneProduct which is degraded. Other parameters are kinetic constants:
- k1 is enzyme-substrate complex formation
- k1r is enzyme-substrate complex degradation
- k2 is rate of substrate degradation when it is in enzyme-substrate complex.

In [ ]:
# Create and add reporter to genetic network
c14 = lc.Regulator(name='C14', color='green', degradation_rate=0, init_concentration=5)
aiia = lc.Regulator(name='AiiA', init_concentration=2)
genetic_n1.add_regulator([c14, aiia])

# Degradation of c14 by lactonase
lactonase = lc.Degrader(name='Lactonase', enzyme=aiia, substrate=c14, k1=1, k1r=0.0001, k2=0.1)
genetic_n1.add_impactor(lactonase)

In [ ]:
sample1 = lc.Sample(strain=test_strain_1)
assay1 = lc.Assay([sample1], 
                n_measurements=100, 
                interval=0.24,
                name='Degradation of C14',
                description='Simulated C14 degradation by lactonase AiiA'
                )

In [ ]:
assay1.run(stochastic=False, track_all=True)

In [ ]:
m = assay1.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='C14 in Test strain'].plot(x='Time', y='Measurement', color='green', style='-', ax=ax)
m[m.Signal=='AiiA in Test strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
plt.legend(['C14', 'AiiA'])


## Producer

First, create metabolism and genetic network, then create strain with these parameters.

In [ ]:
metab2 = lc.SimulatedMetabolism("Simulated metabolism 2", biomass, growth_rate)
genetic_n2 = lc.GeneticNetwork()
test_strain_2= lc.Strain('Test strain', genetic_n2, metab2)

Producer takes the name (use enzyme function name), enzyme is GeneProduct which has the active site, product is the GeneProduct which is synthesized by the enzyme. 
- k2 is kinetic parameter of product synthesis by enzyme-substrate complex.

Here I assume that there is always enough substrate in the system, so all active sites are occupied with substrate at every moment. Because of that, we only need to use k2 kinetic parameter.

In [ ]:
# Create and add reporter to genetic network
c4 = lc.Regulator(name='C4')
rhli = lc.Regulator(name='RhlI', init_concentration=5)
genetic_n2.add_regulator([c4, rhli])

# Synthesis of C4 by RhlI
c4_synthase = lc.Producer(name='C4-HSL synthase', enzyme=rhli, product=c4, k2=0.267)
genetic_n2.add_impactor(c4_synthase)

I have calculated k2 by using data from supplementary text from <div class="csl-entry">Pai, A., &#38; You, L. (2009). Optimal tuning of bacterial sensing potential. <i>Molecular Systems Biology</i>, <i>5</i>, 286. https://doi.org/10.1038/MSB.2009.43</div>

I used 16 mol of C4 produced per mol of RhlI per minute, divded it by 60 to get measurement in seconds.

In [ ]:
sample2 = lc.Sample(strain=test_strain_2)
assay2= lc.Assay([sample2], 
                n_measurements=100, 
                interval=0.24,
                name='Synthesis of C4',
                description='Simulated C4 synthesis by RhlI'
                )

In [ ]:
assay2.run(stochastic=False, track_all=True)

In [ ]:
m = assay2.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='C4 in Test strain'].plot(x='Time', y='Measurement', color='green', style='-', ax=ax)
m[m.Signal=='RhlI in Test strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
plt.legend(['C4', 'RhlI'])
